# 🛠 Kafka Topic Management with confluent-kafka-python
This notebook demonstrates how to manage Kafka topics using Confluent's `confluent-kafka-python` AdminClient API.

## Operations:
- Create a topic
- List topics
- Describe topic metadata
- Increase partitions
- Delete a topic

In [1]:
!pip install -q confluent-kafka

In [4]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions

# Configuration
bootstrap_servers = "kafka.analytics.svc.cluster.local:9092"
admin_client = AdminClient({'bootstrap.servers': bootstrap_servers})

## ✅ Create Topic

In [5]:
topic_name = "my-new-topic"
new_topic = NewTopic(topic=topic_name, num_partitions=3, replication_factor=1)
fs = admin_client.create_topics([new_topic])

for topic, f in fs.items():
    try:
        f.result()
        print(f"Topic '{topic}' created.")
    except Exception as e:
        print(f"Failed to create topic '{topic}': {e}")

Topic 'my-new-topic' created.


## 📜 List Topics

In [6]:
metadata = admin_client.list_topics(timeout=10)
print("Topics:")
for topic in metadata.topics:
    print(f" - {topic}")

Topics:
 - test-topic
 - my-new-topic
 - __consumer_offsets


## 🔍 Describe Topic Metadata

In [7]:
topic = metadata.topics.get(topic_name)
if topic:
    for partition_id, partition in topic.partitions.items():
        print(f"Partition {partition_id}: Leader = {partition.leader}, Replicas = {partition.replicas}")

Partition 0: Leader = 1, Replicas = [1]
Partition 1: Leader = 1, Replicas = [1]
Partition 2: Leader = 1, Replicas = [1]


## ➕ Increase Partitions

In [8]:
# Increase to 6 partitions
fs = admin_client.create_partitions([NewPartitions(topic_name, 6)])
for topic, f in fs.items():
    try:
        f.result()
        print(f"Partitions increased for topic '{topic}'.")
    except Exception as e:
        print(f"Failed to increase partitions: {e}")

Partitions increased for topic 'my-new-topic'.


## 🗑️ Delete Topic

In [9]:
fs = admin_client.delete_topics([topic_name], operation_timeout=30)
for topic, f in fs.items():
    try:
        f.result()
        print(f"Topic '{topic}' deleted.")
    except Exception as e:
        print(f"Failed to delete topic '{topic}': {e}")

Topic 'my-new-topic' deleted.


In [ ]:
import subprocess
subprocess.run([
    "/opt/kafka/bin/kafka-configs.sh", ## fix path if necessary
    "--bootstrap-server", bootstrap_servers,
    "--alter", "--entity-type", "topics",
    "--entity-name", topic_name,
    "--add-config", "retention.ms=60000"
])


FileNotFoundError: [Errno 2] No such file or directory: '/opt/kafka/bin/kafka-configs.sh'